## 第一步：去除多余的信息
- 只保留source和target_id
## 第二步：reshape，做一个target_id命中表 如果target_id命中 就是1 如果target_id没有命中，就是0
- 统计technique表中所有的target_id，并将其放在一个列表中
- 将所有的在target_id列表中的数据作为一个新列
- 如果已有数据的target_id在新列里，就填充1.如果没有，就填充0

In [16]:
import pandas as pd
# 读取整个 Excel 文件
xls = pd.ExcelFile("./DataSource/new_attck_relationship.xlsx")

In [17]:
# 获取所有子表的名称
sheet_names = xls.sheet_names

In [18]:
#打印所有获得子表的名称, 以列表形式存储
print(sheet_names)

['group_use_technique', 'technique', 'group_use_software', 'software_use_technique']


In [19]:
# 选择要读取的特定子表——technique
selected_sheet = sheet_names[1]  # 假设选择第一个子表

# 读取选择的子表数据
df_technique = pd.read_excel(xls, selected_sheet)

In [20]:
# 获取 ID 列的所有数据并放入列表
id_list = df_technique['ID'].tolist()

In [21]:
print(id_list)

['T1548', 'T1548.002', 'T1548.004', 'T1548.001', 'T1548.003', 'T1134', 'T1134.002', 'T1134.003', 'T1134.004', 'T1134.005', 'T1134.001', 'T1531', 'T1087', 'T1087.004', 'T1087.002', 'T1087.003', 'T1087.001', 'T1098', 'T1098.001', 'T1098.003', 'T1098.002', 'T1098.005', 'T1098.004', 'T1583', 'T1583.005', 'T1583.002', 'T1583.001', 'T1583.004', 'T1583.007', 'T1583.003', 'T1583.006', 'T1595', 'T1595.001', 'T1595.002', 'T1595.003', 'T1557', 'T1557.002', 'T1557.003', 'T1557.001', 'T1071', 'T1071.004', 'T1071.002', 'T1071.003', 'T1071.001', 'T1010', 'T1560', 'T1560.003', 'T1560.002', 'T1560.001', 'T1123', 'T1119', 'T1020', 'T1020.001', 'T1197', 'T1547', 'T1547.014', 'T1547.002', 'T1547.006', 'T1547.008', 'T1547.015', 'T1547.010', 'T1547.012', 'T1547.007', 'T1547.001', 'T1547.005', 'T1547.009', 'T1547.003', 'T1547.004', 'T1547.013', 'T1037', 'T1037.002', 'T1037.001', 'T1037.003', 'T1037.004', 'T1037.005', 'T1217', 'T1176', 'T1185', 'T1110', 'T1110.004', 'T1110.002', 'T1110.001', 'T1110.003', 'T16

In [9]:
# 选择要读取的特定子表——group_use_technique
selected_sheet = sheet_names[0]  # 假设选择第一个子表

# 读取选择的子表数据
df_group_technique = pd.read_excel(xls, selected_sheet)

In [10]:
print(df_group_technique)

     source ID  target ID
0        G0099      T1105
1        G0099  T1204.002
2        G0099  T1036.004
3        G0099      T1571
4        G0099      T1027
...        ...        ...
3035     G0045  T1588.002
3036     G0045      T1199
3037     G0045      T1078
3038     G0045  T1059.003
3039     G0045      T1047

[3040 rows x 2 columns]


In [15]:
# 创建一个空字典用于构建JSON数据
json_data = {}

# 遍历每个group
for group in df_group_technique['source ID'].unique():
    # 获取当前group的数据
    group_data = df_group_technique[data['source ID'] == group]

    # 构建group的技术列表
    techniques = group_data['target ID'].tolist()

    # 将group和对应的技术列表添加到JSON字典中
    json_data[group] = techniques

# 将JSON字典转换为JSON字符串
json_str = pd.io.json.dumps(json_data, indent=4)

# 打印JSON字符串
print(json_str)

{
    "G0099":[
        "T1105",
        "T1204.002",
        "T1036.004",
        "T1571",
        "T1027",
        "T1053.005",
        "T1566.001",
        "T1588.002",
        "T1059.005"
    ],
    "G0006":[
        "T1560.001",
        "T1119",
        "T1005",
        "T1584.001",
        "T1583.001",
        "T1585.002",
        "T1003.001",
        "T1087.001",
        "T1114.001",
        "T1588.001",
        "T1036.005",
        "T1135",
        "T1550.002",
        "T1057",
        "T1021.001",
        "T1114.002",
        "T1566.001",
        "T1566.002",
        "T1016",
        "T1049",
        "T1007",
        "T1588.002",
        "T1059.003"
    ],
    "G0005":[
        "T1102.002",
        "T1568.003",
        "T1203",
        "T1204.002",
        "T1566.001"
    ],
    "G0023":[
        "T1584.004"
    ],
    "G0025":[
        "T1585",
        "T1583.006"
    ],
    "G0026":[
        "T1053.002",
        "T1071.004",
        "T1133",
        "T1070.004",
        "T10

保存一个新的df对象，只保留组名和后面对应的technique_id列

In [31]:
# 创建一个空的DataFrame作为新的Excel表格
df = pd.DataFrame(columns=['source ID'] + id_list)

In [33]:
# 创建空行，所有technique_id种类的列都填充为0
empty_row = {column: 0 for column in id_list}

In [34]:
print(df)

Empty DataFrame
Columns: [source ID, T1548, T1548.002, T1548.004, T1548.001, T1548.003, T1134, T1134.002, T1134.003, T1134.004, T1134.005, T1134.001, T1531, T1087, T1087.004, T1087.002, T1087.003, T1087.001, T1098, T1098.001, T1098.003, T1098.002, T1098.005, T1098.004, T1583, T1583.005, T1583.002, T1583.001, T1583.004, T1583.007, T1583.003, T1583.006, T1595, T1595.001, T1595.002, T1595.003, T1557, T1557.002, T1557.003, T1557.001, T1071, T1071.004, T1071.002, T1071.003, T1071.001, T1010, T1560, T1560.003, T1560.002, T1560.001, T1123, T1119, T1020, T1020.001, T1197, T1547, T1547.014, T1547.002, T1547.006, T1547.008, T1547.015, T1547.010, T1547.012, T1547.007, T1547.001, T1547.005, T1547.009, T1547.003, T1547.004, T1547.013, T1037, T1037.002, T1037.001, T1037.003, T1037.004, T1037.005, T1217, T1176, T1185, T1110, T1110.004, T1110.002, T1110.001, T1110.003, T1612, T1115, T1580, T1538, T1526, T1619, T1059, T1059.002, T1059.007, T1059.008, T1059.001, T1059.006, T1059.004, T1059.005, T1059.00

In [35]:
# 遍历JSON数据中的每个group和对应的technique_id列表
for group, techniques in json_data.items():
    # 复制空行，用作当前group的一行数据
    row = empty_row.copy()
    row['source ID'] = group
    
    # 在每个technique_id种类列中填充1
    for technique in techniques:
        if technique in id_list:
            row[technique] = 1
    
    # 将当前行添加到结果DataFrame中
    df = df.append(row, ignore_index=True)

C:\Users\Kerberos233\AppData\Local\Temp\ipykernel_8748\3328707809.py:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(row, ignore_index=True)
C:\Users\Kerberos233\AppData\Local\Temp\ipykernel_8748\3328707809.py:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(row, ignore_index=True)
C:\Users\Kerberos233\AppData\Local\Temp\ipykernel_8748\3328707809.py:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(row, ignore_index=True)
C:\Users\Kerberos233\AppData\Local\Temp\ipykernel_8748\3328707809.py:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(row, ignore_index=True)
C:\Users\Ker

C:\Users\Kerberos233\AppData\Local\Temp\ipykernel_8748\3328707809.py:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(row, ignore_index=True)
C:\Users\Kerberos233\AppData\Local\Temp\ipykernel_8748\3328707809.py:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(row, ignore_index=True)
C:\Users\Kerberos233\AppData\Local\Temp\ipykernel_8748\3328707809.py:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(row, ignore_index=True)
C:\Users\Kerberos233\AppData\Local\Temp\ipykernel_8748\3328707809.py:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(row, ignore_index=True)
C:\Users\Ker

C:\Users\Kerberos233\AppData\Local\Temp\ipykernel_8748\3328707809.py:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(row, ignore_index=True)
C:\Users\Kerberos233\AppData\Local\Temp\ipykernel_8748\3328707809.py:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(row, ignore_index=True)
C:\Users\Kerberos233\AppData\Local\Temp\ipykernel_8748\3328707809.py:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(row, ignore_index=True)
C:\Users\Kerberos233\AppData\Local\Temp\ipykernel_8748\3328707809.py:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(row, ignore_index=True)
C:\Users\Ker

C:\Users\Kerberos233\AppData\Local\Temp\ipykernel_8748\3328707809.py:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(row, ignore_index=True)
C:\Users\Kerberos233\AppData\Local\Temp\ipykernel_8748\3328707809.py:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(row, ignore_index=True)
C:\Users\Kerberos233\AppData\Local\Temp\ipykernel_8748\3328707809.py:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(row, ignore_index=True)
C:\Users\Kerberos233\AppData\Local\Temp\ipykernel_8748\3328707809.py:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(row, ignore_index=True)
C:\Users\Ker

In [36]:
print(df)

    source ID T1548 T1548.002 T1548.004 T1548.001 T1548.003 T1134 T1134.002  \
0       G0099     0         0         0         0         0     0         0   
1       G0006     0         0         0         0         0     0         0   
2       G0005     0         0         0         0         0     0         0   
3       G0023     0         0         0         0         0     0         0   
4       G0025     0         0         0         0         0     0         0   
..        ...   ...       ...       ...       ...       ...   ...       ...   
125     G0044     0         0         0         0         0     0         0   
126     G0102     0         0         0         0         0     0         0   
127     G0128     0         0         0         0         0     0         0   
128     G0018     0         0         0         0         0     0         0   
129     G0045     0         0         0         0         0     0         0   

    T1134.003 T1134.004  ... T1497.002 T1600 T1600.

In [38]:
# 将结果保存到Excel表格
df.to_excel('./DataSource/result.xlsx', index=False)

In [20]:
# 创建一个空的 DataFrame，用于存储原有表的数据和新建的列
new_df = pd.DataFrame()

# 将原有表的数据复制到新的 DataFrame
new_df = df_group_technique.copy()

# 根据目标ID列表创建对应的空列
for target_id in id_list:
    new_df.insert(len(new_df.columns), target_id, '')

# 打印新的 DataFrame
print(new_df)

C:\Users\Kerberos233\AppData\Local\Temp\ipykernel_1924\2660021172.py:9: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  new_df.insert(len(new_df.columns), target_id, '')
C:\Users\Kerberos233\AppData\Local\Temp\ipykernel_1924\2660021172.py:9: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  new_df.insert(len(new_df.columns), target_id, '')
C:\Users\Kerberos233\AppData\Local\Temp\ipykernel_1924\2660021172.py:9: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which ha

     source ID  target ID T1548 T1548.002 T1548.004 T1548.001 T1548.003 T1134  \
0        G0099      T1105                                                       
1        G0099  T1204.002                                                       
2        G0099  T1036.004                                                       
3        G0099      T1571                                                       
4        G0099      T1027                                                       
...        ...        ...   ...       ...       ...       ...       ...   ...   
3035     G0045  T1588.002                                                       
3036     G0045      T1199                                                       
3037     G0045      T1078                                                       
3038     G0045  T1059.003                                                       
3039     G0045      T1047                                                       

     T1134.002 T1134.003  .

C:\Users\Kerberos233\AppData\Local\Temp\ipykernel_1924\2660021172.py:9: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  new_df.insert(len(new_df.columns), target_id, '')
C:\Users\Kerberos233\AppData\Local\Temp\ipykernel_1924\2660021172.py:9: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  new_df.insert(len(new_df.columns), target_id, '')
C:\Users\Kerberos233\AppData\Local\Temp\ipykernel_1924\2660021172.py:9: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which ha

In [59]:
# 获取 ID 列的所有数据并放入列表
target_id_list = df_group_technique['target ID'].tolist()

In [60]:
print(target_id_list)

['T1105', 'T1204.002', 'T1036.004', 'T1571', 'T1027', 'T1053.005', 'T1566.001', 'T1588.002', 'T1059.005', 'T1560.001', 'T1119', 'T1005', 'T1584.001', 'T1583.001', 'T1585.002', 'T1003.001', 'T1087.001', 'T1114.001', 'T1588.001', 'T1036.005', 'T1135', 'T1550.002', 'T1057', 'T1021.001', 'T1114.002', 'T1566.001', 'T1566.002', 'T1016', 'T1049', 'T1007', 'T1588.002', 'T1059.003', 'T1102.002', 'T1568.003', 'T1203', 'T1204.002', 'T1566.001', 'T1584.004', 'T1585', 'T1583.006', 'T1053.002', 'T1071.004', 'T1133', 'T1070.004', 'T1083', 'T1105', 'T1027', 'T1547.001', 'T1082', 'T1078', 'T1071.001', 'T1059.003', 'T1059', 'T1574.002', 'T1140', 'T1189', 'T1564.003', 'T1204.002', 'T1112', 'T1027', 'T1059.001', 'T1547.001', 'T1218.010', 'T1218.011', 'T1566.001', 'T1132.001', 'T1082', 'T1016', 'T1033', 'T1588.002', 'T1071.001', 'T1543.003', 'T1098.002', 'T1550.001', 'T1560', 'T1560.001', 'T1119', 'T1102.002', 'T1542.003', 'T1110', 'T1070.001', 'T1078.004', 'T1092', 'T1546.015', 'T1589.001', 'T1030', 'T121

In [58]:
# 创建一个 ExcelWriter 对象，用于写入 Excel 文件
excel_writer = pd.ExcelWriter('./DataSource/test.xlsx', engine='xlsxwriter')

# 将 new_df 写入指定的子表
new_df.to_excel(excel_writer, sheet_name='group_use_technique', index=False)

# 保存 Excel 文件
excel_writer.save()

C:\Users\Kerberos233\AppData\Local\Temp\ipykernel_9596\4235573611.py:8: FutureWarning: save is not part of the public API, usage can give unexpected results and will be removed in a future version
  excel_writer.save()


将处理好的表格形式存入excel中，然后重新读取

In [2]:
import pandas as pd

# 读取数据文件
data = pd.read_excel("./DataSource/test.xlsx")  # 替换为你的数据文件路径
# 总的技术种类列
total_techniques = data.columns[2:]  # 假设总的技术种类列从第4列开始

print(total_techniques)

Index(['T1548', 'T1548.002', 'T1548.004', 'T1548.001', 'T1548.003', 'T1134',
       'T1134.002', 'T1134.003', 'T1134.004', 'T1134.005',
       ...
       'T1497.002', 'T1600', 'T1600.002', 'T1600.001', 'T1102', 'T1102.002',
       'T1102.001', 'T1102.003', 'T1047', 'T1220'],
      dtype='object', length=594)


In [5]:
# 遍历每一行数据
for index, row in data.iterrows():
    # 获取当前行的target id，是单个变量
    target_id = row["target ID"]
    # 设置索引
    index_column = 0
    # 遍历总的技术种类列
    for column in total_techniques:
        # 判断target id下的技术种类是否存在于总的技术种类列中
        if target_id in column:
            # 在存在的技术种类列中填充1
            # print(data.iloc[index, index_column+3])
            data.iloc[index, index_column+3] = 1
        else:
            # 在不存在的技术种类列中填充0
            # print(data.iloc[index, index_column+3])
            data.iloc[index, index_column+3] = 0
        index_column = index_column + 1

T1105
0
T1204.002
0
T1036.004
0
T1571
0
T1027
0
T1027.001
0
T1027.004
0
T1027.007
0
T1027.009
0
T1027.006
0
T1027.005
0
T1027.002
0
T1027.003
0
T1027.008
0
T1053.005
0
T1566.001
0
T1588.002
0
T1059.005
0
T1560.001
0
T1119
0
T1005
0
T1584.001
0
T1583.001
0
T1585.002
0
T1003.001
0
T1087.001
0
T1114.001
0
T1588.001
0
T1036.005
0
T1135
0
T1550.002
0
T1057
0
T1021.001
0
T1114.002
0
T1566.001
0
T1566.002
0
T1016
0
T1016.001
0
T1049
0
T1007
0
T1588.002
0
T1059.003
0
T1102.002
0
T1568.003
0
T1203
0
T1204.002
0
T1566.001
0
T1584.004
0
T1585
0
T1585.003
0
T1585.002
0
T1585.001
0
T1583.006
0
T1053.002
0
T1071.004
0
T1133
0
T1070.004
0
T1083
0
T1105
0
T1027
0
T1027.001
0
T1027.004
0
T1027.007
0
T1027.009
0
T1027.006
0
T1027.005
0
T1027.002
0
T1027.003
0
T1027.008
0
T1547.001
0
T1082
0
T1078
0
T1078.004
0
T1078.001
0
T1078.002
0
T1078.003
0
T1071.001
0
T1059.003
0
T1059
0
T1059.002
0
T1059.007
0
T1059.008
0
T1059.001
0
T1059.006
0
T1059.004
0
T1059.005
0
T1059.003
0
T1574.002
0
T1140
0
T1189
0
T156

T1021.001
0
T1053.005
0
T1033
0
T1588.002
0
T1078
0
T1078.004
0
T1078.001
0
T1078.002
0
T1078.003
0
T1059.003
0
T1564.008
0
T1056.002
0
T1056.001
0
T1204.002
0
T1204.001
0
T1090.003
0
T1114.002
0
T1566.001
0
T1566.002
0
T1078
0
T1078.004
0
T1078.001
0
T1078.002
0
T1078.003
0
T1059.005
0
T1071.001
0
T1119
0
T1110
0
T1110.004
0
T1110.002
0
T1110.001
0
T1110.003
0
T1070.001
0
T1059
0
T1059.002
0
T1059.007
0
T1059.008
0
T1059.001
0
T1059.006
0
T1059.004
0
T1059.005
0
T1059.003
0
T1090.002
0
T1133
0
T1070.004
0
T1074.001
0
T1018
0
T1588.002
0
T1078
0
T1078.004
0
T1078.001
0
T1078.002
0
T1078.003
0
T1134
0
T1134.002
0
T1134.003
0
T1134.004
0
T1134.005
0
T1134.001
0
T1560
0
T1560.003
0
T1560.002
0
T1560.001
0
T1560.003
0
T1573.002
0
T1119
0
T1553.002
0
T1059
0
T1059.002
0
T1059.007
0
T1059.008
0
T1059.001
0
T1059.006
0
T1059.004
0
T1059.005
0
T1059.003
0
T1555
0
T1555.003
0
T1555.001
0
T1555.005
0
T1555.002
0
T1555.004
0
T1555.003
0
T1213
0
T1213.003
0
T1213.001
0
T1213.002
0
T1005
0
T1562.00

T1090.002
0
T1070.004
0
T1105
0
T1059.007
0
T1003.001
0
T1204.002
0
T1036.005
0
T1112
0
T1106
0
T1571
0
T1027
0
T1027.001
0
T1027.004
0
T1027.007
0
T1027.009
0
T1027.006
0
T1027.005
0
T1027.002
0
T1027.003
0
T1027.008
0
T1059.001
0
T1055
0
T1055.004
0
T1055.001
0
T1055.011
0
T1055.015
0
T1055.002
0
T1055.009
0
T1055.013
0
T1055.012
0
T1055.008
0
T1055.003
0
T1055.005
0
T1055.014
0
T1547.001
0
T1021.001
0
T1018
0
T1053.005
0
T1113
0
T1569.002
0
T1072
0
T1566.001
0
T1588.002
0
T1078
0
T1078.004
0
T1078.001
0
T1078.002
0
T1078.003
0
T1125
0
T1059.005
0
T1059.003
0
T1588.004
0
T1583.001
0
T1585.002
0
T1589.002
0
T1114
0
T1114.003
0
T1114.001
0
T1114.002
0
T1114.003
0
T1589.003
0
T1608.005
0
T1110.003
0
T1594
0
T1598.003
0
T1588.002
0
T1078
0
T1078.004
0
T1078.001
0
T1078.002
0
T1078.003
0
T1071.002
0
T1071.003
0
T1071.001
0
T1560.001
0
T1039
0
T1083
0
T1056.001
0
T1036.005
0
T1135
0
T1003
0
T1003.008
0
T1003.005
0
T1003.006
0
T1003.004
0
T1003.001
0
T1003.003
0
T1003.007
0
T1003.002
0
T108

能不能读取表格，然后把group对应的technique存成json数据，
然后再从json数据里面重新创建一张表格，
如果在后序列中存在填1，在后序列中不存在填0

In [ ]:
# 遍历每个group
for group, techniques in json_str.items():
    # 遍历每个technique_id
    for technique in techniques:
        # 遍历后续的technique_id列
        for column in total_techniques:
            # 判断technique_id是否在后续的technique_id列中
            if technique in column:
                # 在存在的technique_id列中填充1
                result.loc[result['source ID'] == group, column] = 1
            else:
                # 在不存在的technique_id列中填充0
                result.loc[result['source ID'] == group, column] = 0

In [13]:
# 保存修改后的数据
data.to_excel("./DataSource/test_2.xlsx", index=False)  # 替换为你想保存的文件路径，index=False表示不保存索引